# CASE - Petlove - Engenharia de Dados
## Valéria Andrade Calado Dantas

Neste case apresento minha análise e sugestões visando solucionar o desafio da equipe de Assinatura da Petlove: reduzir a taxa de Churn, ou seja, a perda de assinantes do serviço de assinatura.

Ao analisar os dados dos últimos meses, observei que, apesar das melhorias de usabilidade da plataforma, o Churn tem apresentado um aumento preocupante. Com base nessas informações, apliquei técnicas de Engenharia, Ciência e Análise de Dados, a fim de identificar os principais aprendizados e sugerir próximos passos. Durante o processo, busquei compreender os motivos subjacentes ao aumento do Churn e identificar padrões e tendências nos dados disponíveis. Através dessa análise, foram obtidos insights que podem auxiliar a equipe de Assinatura a tomar decisões estratégicas.

Através dessa resposta, espero demonstrar minha capacidade de aplicar técnicas de Engenharia de Dados para solucionar problemas reais, contribuindo para o sucesso da equipe de Assinatura da Petlove.


## Análise de Churn: Filtros Estratégicos para Identificar Padrões e Tendências

### 1. Clientes com Assinatura Pausada
Inicialmente, foi realizada uma análise para verificar a quantidade de clientes em diferentes estados de assinatura: ativos, pausados e cancelados. Essa ação teve como objetivo compreender a distribuição dos clientes. Foi verificado que 8.524 clientes estão ativos, 971 estão com a assinatura pausada e 505 clientes cancelaram o serviço.

In [ ]:
#@title 
import pandas as pd

df = pd.read_csv("/content/data-test-analytics_5.csv")

paused = df['status'] == 'paused'
paused = paused.sort_values(ascending=False)
sum(paused)

active = df['status'] == 'active'
active = active.sort_values(ascending=False)
sum(paused)

canceled = df['status'] == 'canceled'
canceled = canceled.sort_values(ascending=False)
sum(paused)

In [ ]:
#@title 
import matplotlib.pyplot as plt

quantidades = df['status'].value_counts()

plt.bar(quantidades.index, quantidades.values, width=0.2)

plt.xlabel('Status')
plt.ylabel('Quantidade')
plt.title('Quantidades de clientes ativos, pausados e cancelados')

for i, value in enumerate(quantidades.values):
    plt.annotate(str(value), xy=(i, value), ha='center', va='bottom')

plt.show()

<img src="https://i.ibb.co/D4WtfP7/grafico-pausados.png">

Quando os clientes pausam suas assinaturas, isso pode indicar que eles estão enfrentando algum tipo de problema, insatisfação ou simplesmente querem interromper temporariamente o uso do serviço. Monitorar a quantidade de clientes pausados permite identificar padrões e tendências, fornecendo a oportunidade de intervir e resolver possíveis problemas, antes que os clientes decidam cancelar definitivamente.
Os clientes pausados representam uma oportunidade de recuperação. Eles já demonstraram interesse no serviço e podem estar mais propensos a retomar suas assinaturas se os problemas ou preocupações forem abordados de forma eficaz.

### 2. Tempo desde a última compra do cliente:

Foi realizado um filtro para identificar os clientes que não fizeram compras há mais de 45 dias e foi obtido o total de 111 clientes. Essa análise é útil para identificar potenciais casos de churn, ou seja, clientes que podem estar em risco de cancelar a assinatura ou deixar de utilizar os serviços.

A definição desse limite de tempo de 45 dias é uma estratégia para identificar clientes que estão inativos por um período considerável e essa informação pode ser valiosa para ações de reengajamento.

In [ ]:
#@title 
data_referencia = pd.Timestamp('2021-02-16')

limite_tempo = data_referencia - pd.DateOffset(days=45)

df['last_date_purchase'] = pd.to_datetime(df['last_date_purchase'])

churn = df[(df['last_date_purchase'] < limite_tempo) & (df['status'] == 'active')]

len(churn)

### 3. Queda no valor médio de gasto por pedido:

Nesse passo, foram identificados os clientes com uma queda no valor médio de gasto ao longo do tempo desde a criação da assinatura até a data de análise. Esse filtro é importante para verificar clientes estão gastando menos com o passar do tempo, o que pode ser um sinal de desinteresse, insatisfação com o serviço ou até mesmo uma possível intenção de cancelamento no futuro.

In [ ]:
#@title 
data_analise = pd.to_datetime('2021-02-16')

clientes_ativos = df[df['status'] == 'active'].copy()

clientes_ativos['created_at'] = pd.to_datetime(clientes_ativos['created_at'])

clientes_ativos['Media_Gasto_Desde_Criacao'] = clientes_ativos.apply(lambda row: clientes_ativos[(clientes_ativos['created_at'] <= row['created_at']) & (clientes_ativos['created_at'] <= data_analise)]['average_ticket'].mean(), axis=1)

clientes_queda = clientes_ativos[clientes_ativos['average_ticket'] < clientes_ativos['Media_Gasto_Desde_Criacao']]

clientes_queda.head(1000)

### 4. Churn por Estado:
Aqui foi verificada a quantidade de churn por Estado brasileiro, fornecendo insights sobre a distribuição geográfica dos cancelamentos de assinaturas. Isso pode ser útil para identificar padrões ou tendências regionais que podem estar influenciando no churn dos clientes.

A análise acerca de quais estados têm maior incidência de cancelamentos pode indicar problemas específicos nesses locais, como por exemplo, pode haver questões relacionadas ao atendimento ao cliente, logística, oferta de serviços ou até mesmo diferenças culturais que estão impactando negativamente a retenção de clientes em determinadas regiões.

In [ ]:
#@title 
import matplotlib.pyplot as plt

cancelados_por_estado = df[df['status'] == 'canceled'].groupby('state').size()
cancelados_por_estado = cancelados_por_estado.sort_values(ascending=False)

tabela_cancelados_por_estado = pd.DataFrame({'Estado': cancelados_por_estado.index, 'Cancelados': cancelados_por_estado.values})

plt.figure(figsize=(10, 6))

estados = tabela_cancelados_por_estado['Estado']
cancelamentos = tabela_cancelados_por_estado['Cancelados']

plt.bar(estados, cancelamentos)

for i in range(len(estados)):
    plt.text(estados[i], cancelamentos[i], str(cancelamentos[i]), ha='center', va='bottom')

plt.title('Churn por Estado')
plt.xlabel('Estado')
plt.ylabel('Número de Cancelamentos')

plt.xticks(rotation=45)

plt.show()

<img src='https://i.ibb.co/3m3fMpq/Unknown-2.png'>

### 5. Churn Por Idade:

A análise da quantidade de cancelamentos por faixa de idade pode fornecer insights valiosos sobre as preferências, necessidades e características dos clientes em cada faixa de idade. É possível identificar as faixas etárias que têm maior propensão a cancelar os serviços e identificar possíveis razões por trás disso.
Além disso, a análise dos cancelamentos por faixa de idade permite segmentar o público-alvo e personalizar as estratégias de marketing e no desenvolvimento de ações específicas para reter clientes em diferentes faixas etárias.

In [ ]:
#@title 
from datetime import datetime

df['birth_date'] = pd.to_datetime(df['birth_date'])

data_atual = datetime.now()

df['idade'] = (data_atual - df['birth_date']).astype('<m8[Y]')

faixas_idade = [0, 18, 25, 35, 45, 55, float('inf')]
labels_faixas = ['0-18', '19-25', '26-35', '36-45', '46-55', '55+']

df['faixa_idade'] = pd.cut(df['idade'], bins=faixas_idade, labels=labels_faixas)

cancelamentos_por_idade = df[df['status'] == 'canceled'].groupby('faixa_idade').size()

print(cancelamentos_por_idade)


In [ ]:
#@title 
import matplotlib.pyplot as plt

cancelados_por_idade = df[df['status'] == 'canceled'].groupby('faixa_idade').size().reset_index(name='Cancelamentos')

tabela_cancelamentos_por_idade = pd.DataFrame(cancelados_por_idade)

plt.figure(figsize=(7, 3))

faixas_idade = tabela_cancelamentos_por_idade['faixa_idade']
cancelamentos = tabela_cancelamentos_por_idade['Cancelamentos']

plt.bar(faixas_idade, cancelamentos)

plt.title('Cancelamentos por Faixa de Idade')
plt.xlabel('Faixa de Idade')
plt.ylabel('Número de Cancelamentos')

plt.xticks(rotation=45)

plt.show()


<img src="https://i.ibb.co/jMkjdN2/Unknown-3.png">

### 6. Receita Perdida em Razão do Churn:
Por último, foi calculada a média da receita dos clientes cancelados, totalizando o valor de R$ 1.059,40.

Esse filtro tem o objetivo de avaliar o impacto financeiro e entender o valor médio que a empresa está perdendo devido aos cancelamentos. Isso ajuda a ter uma visão clara do impacto financeiro e a tomar decisões estratégicas para mitigar as perdas.

In [ ]:
#@title 
clientes_cancelados = df[df['status'] == 'canceled']

media_receita_cancelados = clientes_cancelados['all_revenue'].mean()

print(media_receita_cancelados)

## Próximos Passos

Com base nas informações extraídas do DataFrame, aqui estão alguns insights:

1. Clientes com assinatura pausada:
   - Desenvolver estratégias para reativar os clientes pausados, como ofertas especiais, descontos ou brindes para incentivá-los a retomar a assinatura.
   - Acompanhar a taxa de recorrência de pausas e analisar se há algum padrão sazonal ou recorrente.


2. Clientes que não compram há mais de 45 dias:
   - Realizar campanhas de reengajamento direcionadas aos clientes inativos, oferecendo descontos exclusivos, brindes ou novidades relacionadas aos produtos.
   - Enviar lembretes personalizados aos clientes inativos, destacando os benefícios da assinatura e incentivando-os a retomar as compras.


3. Clientes com queda no valor médio de gasto por pedido:
   - Identificar os fatores que podem ter contribuído para a queda no valor médio de gasto por pedido, como mudança de preferências, insatisfação com os produtos ou problemas de logística.
   - Realizar pesquisas ou enquetes com os clientes para obter feedback sobre a experiência de compra e descobrir maneiras de melhorar a oferta de produtos e serviços.
   - Oferecer incentivos para os clientes aumentarem o valor médio de gasto por pedido, como descontos que recompensem compras maiores.


4. Churn por Estado:
   - Analisar os dados geográficos para identificar possíveis oportunidades de expansão em áreas com baixo churn e alto potencial de crescimento.
   - Compreender as características demográficas e comportamentais dos clientes em diferentes Estados para adaptar as estratégias de marketing e comunicação, levando em consideração as particularidades de cada região.
   - Avaliar se há questões específicas em determinados Estados que podem estar contribuindo para o churn, como problemas logísticos, concorrência ou falta de conhecimento sobre os benefícios da assinatura.
   
   
5. Churn por idade:
   - Analisar as faixas etárias com maior churn para identificar possíveis motivos de insatisfação ou necessidades não atendidas em relação aos produtos.
   - Adaptar as estratégias de marketing e comunicação para melhor atender às necessidades e preferências das diferentes faixas etárias.